In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI()

In [4]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):
    prompt = f"Generate a joke on the topic {state['topic']}"
    response = llm.invoke(prompt).content
    
    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):
    prompt = f"Generate an explanation for this joke: {state['joke']}"
    response = llm.invoke(prompt).content
    
    return {'explanation': response}

In [10]:
# Create a Graph
graph = StateGraph(JokeState)

# Add Nodes
graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explanation", generate_explanation)

# Create Edges
graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "generate_explanation")
graph.add_edge("generate_explanation", END)

# Create In-memory Checkpointer
checkpointer = InMemorySaver()

# Compile the Graph
workflow = graph.compile(checkpointer=checkpointer)

In [12]:
config1 = {"configurable": {'thread_id': '1'}}

initial_state = {
    "topic": "Coffee mug"
}

workflow.invoke(initial_state, config=config1)

{'topic': 'Coffee mug',
 'joke': 'Why did the coffee mug go to the therapist? Because it was feeling a little tea-rse.',
 'explanation': 'This joke plays on the pun between the words "tea-rse" and "teased". The coffee mug went to the therapist because it was feeling a little "tea-rse," meaning it was feeling teased or irritated. The joke creates humor by anthropomorphizing the coffee mug and giving it feelings and emotions like a person would have.'}

In [13]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug go to the therapist? Because it was feeling a little tea-rse.', 'explanation': 'This joke plays on the pun between the words "tea-rse" and "teased". The coffee mug went to the therapist because it was feeling a little "tea-rse," meaning it was feeling teased or irritated. The joke creates humor by anthropomorphizing the coffee mug and giving it feelings and emotions like a person would have.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb5-d45a-60bc-8002-b604ecd51a78'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-25T10:35:32.917045+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb5-c9c3-6194-8001-fd87e353c69b'}}, tasks=(), interrupts=())

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug go to the therapist? Because it was feeling a little tea-rse.', 'explanation': 'This joke plays on the pun between the words "tea-rse" and "teased". The coffee mug went to the therapist because it was feeling a little "tea-rse," meaning it was feeling teased or irritated. The joke creates humor by anthropomorphizing the coffee mug and giving it feelings and emotions like a person would have.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb5-d45a-60bc-8002-b604ecd51a78'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-25T10:35:32.917045+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb5-c9c3-6194-8001-fd87e353c69b'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug go to the therapist? Because it was feeling a little tea

In [16]:
workflow.invoke({'topic':'Flying Cars'}, config=config1)

{'topic': 'Flying Cars',
 'joke': "Why did the flying car break up with the airplane? \nBecause it couldn't handle the heights of the relationship!",
 'explanation': 'This joke plays on the idea that the flying car and the airplane were in a romantic relationship. The punchline reveals that the flying car broke up with the airplane because it couldn\'t handle the "heights" of the relationship, both figuratively referring to the emotional intensity and challenges of the relationship and literally referring to the fact that airplanes fly at high altitudes. It\'s a light-hearted way to play on words and create a humorous scenario.'}

In [17]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Flying Cars', 'joke': "Why did the flying car break up with the airplane? \nBecause it couldn't handle the heights of the relationship!", 'explanation': 'This joke plays on the idea that the flying car and the airplane were in a romantic relationship. The punchline reveals that the flying car broke up with the airplane because it couldn\'t handle the "heights" of the relationship, both figuratively referring to the emotional intensity and challenges of the relationship and literally referring to the fact that airplanes fly at high altitudes. It\'s a light-hearted way to play on words and create a humorous scenario.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb9-5d54-67ac-8006-d93377d07385'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T10:37:07.810895+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb9-50fe-6b2e-

In [18]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Flying Cars', 'joke': "Why did the flying car break up with the airplane? \nBecause it couldn't handle the heights of the relationship!", 'explanation': 'This joke plays on the idea that the flying car and the airplane were in a romantic relationship. The punchline reveals that the flying car broke up with the airplane because it couldn\'t handle the "heights" of the relationship, both figuratively referring to the emotional intensity and challenges of the relationship and literally referring to the fact that airplanes fly at high altitudes. It\'s a light-hearted way to play on words and create a humorous scenario.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb9-5d54-67ac-8006-d93377d07385'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T10:37:07.810895+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099fb9-50fe-6b2e